# Julia Packages

So far, we've seen
* basic Julia syntax
* the Julia type system

Today, we're going to 
* See some of the features of the package manager
* Demo some packages for numerical linear algebra
* Demo some packages for visualization
* Since you all may have different ideas of what packages you may want to use, we'll try to have some open-ended time for trying out whatever you'd like.  I can (try to) help with any installation issues, or depending on the pacakge, perhaps some basic usage.

Future plan
* Next week: git version control and how to make your own package
* Second half of the class: "Advanced Topics" portion of the course

## Packages We May Use Today

* LinearOperators
* IterativeSolvers
* DataFrames
* Plots
* ProgressMeter
* StatPlots
* RDatasets
* GR
* PyPlot
* PlotlyJS
* Interact

Go ahead and ```Pkg.add``` them in Julia.


# Packages

(This IJulia notebook is tested using Julia [v0.6.2](https://github.com/JuliaLang/julia/releases))

Julia currently has [over 1600 packages](http://pkg.julialang.org/pulse.html) registered.  This includes relatively mature and popular pacakges as well as packages that are just getting off the ground.

Today we'll look a bit more at the package manager and some popular Julia packages.

You have already used [Pkg.add()](http://julia.readthedocs.org/en/latest/stdlib/pkg/#Base.Pkg.add) to start using packages such as IJulia and PyPlot.  You can also add a version number to the arguments to use a particular version of a package.  You can use [Pkg.available()](http://julia.readthedocs.org/en/latest/stdlib/pkg/#Base.Pkg.available) to list available versions of a package.

In [1]:
Pkg.available("PyPlot")

51-element Array{VersionNumber,1}:
 v"0.0.0" 
 v"1.0.0" 
 v"1.0.1" 
 v"1.0.2" 
 v"1.0.3" 
 v"1.0.4" 
 v"1.0.5" 
 v"1.0.6" 
 v"1.0.7" 
 v"1.0.8" 
 v"1.0.9" 
 v"1.0.10"
 v"1.0.11"
 ⋮        
 v"2.0.0" 
 v"2.0.1" 
 v"2.1.0" 
 v"2.1.1" 
 v"2.2.0" 
 v"2.2.1" 
 v"2.2.2" 
 v"2.2.3" 
 v"2.2.4" 
 v"2.3.0" 
 v"2.3.1" 
 v"2.3.2" 

Recall that you can use Pkg.status() to view all your installed packages.  Required pacakges are those that you specifically added.  Additional packages are those that are required as dependencies of the required packages.

In [2]:
Pkg.status()

41 required packages:
 - Atom                          0.6.6
 - AudioIO                       0.1.1
 - Blink                         0.6.0
 - Cbc                           0.3.2
 - Convex                        0.5.0
 - DataFrames                    0.10.1
 - DataStructures                0.7.4
 - GLVisualize                   0.6.1
 - GR                            0.25.0
 - Gadfly                        0.6.4
 - GraphPlot                     0.2.0
 - Graphs                        0.9.0
 - IJulia                        1.6.2
 - Interact                      0.6.3
 - IterTools                     0.2.1
 - IterativeSolvers              0.4.2
 - JLD                           0.8.3
 - JuMP                          0.18.0
 - Laplacians                    0.2.2
 - LightGraphs                   0.11.1
 - LightGraphsExtras             0.1.0
 - LinearOperators               0.3.1
 - MAT                           0.4.0
 - Mosek                         0.8.2
 - MultivariatePolynomials       0.1.2

If you want to see if a particular package is installed, you can use Pkg.installed()

In [3]:
Pkg.installed("IJulia")

v"1.6.2"

Sometimes after updating a package you may need to re-build it.

In [4]:
Pkg.build("Plots")

INFO: Building SpecialFunctions
INFO: Building Plots


If you're having trouble getting packages to play nicely, you can try using [Pkg.resolve()](http://julia.readthedocs.org/en/latest/stdlib/pkg/#Base.Pkg.resolve).

In [5]:
Pkg.resolve()

INFO: No packages to install, update or remove


If you're debugging a package, Pkg.test() will run tests provided with a package

In [6]:
Pkg.test("PyCall")

INFO: Testing PyCall
INFO: Python version 2.7.14 from /usr/lib64/libpython2.7, PYTHONHOME=/usr:/usr
ENV[PYTHONPATH]=:/home/brad/code/gudhi/2.0.1/build/cython:/home/brad/code/gudhi/2.0.1/build/cython
ENV[PYTHONHOME]=
ENV[PYTHONEXECUTABLE]=
INFO: PyCall tests passed


In [ ]:
Pkg.update()

If you want to use a package that is not a registered Julia package or if you want to use the current development branch of a package, use Pkg.clone() with a GitHub URL.

In [9]:
Pkg.clone("git@github.com:JuliaSparse/Metis.jl.git")

INFO: Cloning Metis from git@github.com:JuliaSparse/Metis.jl.git
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package database updated


In [10]:
Pkg.status()

42 required packages:
 - Atom                          0.6.6
 - AudioIO                       0.1.1
 - Blink                         0.6.0
 - Cbc                           0.3.2
 - Convex                        0.5.0
 - DataFrames                    0.10.1
 - DataStructures                0.7.4
 - GLVisualize                   0.6.1
 - GR                            0.25.0
 - Gadfly                        0.6.4
 - GraphPlot                     0.2.0
 - Graphs                        0.9.0
 - IJulia                        1.6.2
 - Interact                      0.6.3
 - IterTools                     0.2.1
 - IterativeSolvers              0.4.2
 - JLD                           0.8.3
 - JuMP                          0.18.0
 - Laplacians                    0.2.2
 - LightGraphs                   0.11.1
 - LightGraphsExtras             0.1.0
 - LinearOperators               0.3.1
 - MAT                           0.4.0
 - Metis                         0.0.10             master
 - Mosek          

In [11]:
Pkg.rm("Metis")

INFO: No packages to install, update or remove
INFO: Package database updated


In [12]:
Pkg.status()

41 required packages:
 - Atom                          0.6.6
 - AudioIO                       0.1.1
 - Blink                         0.6.0
 - Cbc                           0.3.2
 - Convex                        0.5.0
 - DataFrames                    0.10.1
 - DataStructures                0.7.4
 - GLVisualize                   0.6.1
 - GR                            0.25.0
 - Gadfly                        0.6.4
 - GraphPlot                     0.2.0
 - Graphs                        0.9.0
 - IJulia                        1.6.2
 - Interact                      0.6.3
 - IterTools                     0.2.1
 - IterativeSolvers              0.4.2
 - JLD                           0.8.3
 - JuMP                          0.18.0
 - Laplacians                    0.2.2
 - LightGraphs                   0.11.1
 - LightGraphsExtras             0.1.0
 - LinearOperators               0.3.1
 - MAT                           0.4.0
 - Mosek                         0.8.2
 - MultivariatePolynomials       0.1.2

# Linear Operators & Iterative Solvers

Go to the [iterative solvers notebook](https://github.com/icme/cme257-advanced-julia/blob/master/extras/iterative_solvers.ipynb) on the class website in the extras folder.

# Data and Plots

Go to the [data and plots notebook](https://github.com/icme/cme257-advanced-julia/blob/master/extras/data_and_plots.ipynb) on the class website in the extras folder.

# Rest of Class

Try installing/using any package you think is interesting.  Check out the [package listing](https://pkg.julialang.org/), or clone a package that is in development.

# Final Notes

* No HW this week
* Next week, we'll talk about git version control, and making your own package
* HW next week will be to make an ipynb demoing a Julia package of your choice, and creating a pull request to the class Julia repository to share it.  You can start on the ipynb ahead of time if you'd like.